<a href="https://colab.research.google.com/github/amsac/ML_Notebooks/blob/main/lc/Langchain_Router_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LangChain 🦜🔗:  Routing**


### **[Reference](https://python.langchain.com/docs/how_to/routing/)**

In [ ]:
# Install the OpenAI package, which allows access to OpenAI models (e.g., GPT-3, GPT-4)
!pip install openai

# Install LangChain, a framework for building applications using LLMs (Large Language Models)
!pip install langchain

# Install the LangChain OpenAI integration package to connect LangChain with OpenAI models
!pip install langchain-openai

In [ ]:
# Import the OpenAI Python client library, which allows us to interact with OpenAI models
import openai

# Import the OS module to interact with the operating system, like managing environment variables
import os

In [ ]:
# Open the file containing the OpenAI API key
f = open('/content/ts_openapi_key.txt')

# Read the API key from the file and store it in the 'api_key' variable
api_key = f.read()

# Set the API key as an environment variable, so OpenAI can access it for authentication
os.environ['OPENAI_API_KEY'] = api_key

# Assign the environment variable value to OpenAI's API key attribute
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
# Import the ChatOpenAI class from langchain_openai, which provides a wrapper for interacting with OpenAI's chat models
from langchain_openai import ChatOpenAI

# Import the PromptTemplate class from langchain_core.prompts, which is used to create prompt templates
from langchain_core.prompts import PromptTemplate

# Import the ChatPromptTemplate class, which helps in creating chat-based prompt templates
from langchain_core.prompts import ChatPromptTemplate

# Import the StrOutputParser from langchain_core.output_parsers, which is used to parse outputs as strings
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Specify the model name for the chat model you want to use
llm_model = "gpt-4o-mini"  # This is a chat model

# Initialize the ChatOpenAI model with specific parameters
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
# Define a PromptTemplate to classify user questions into categories
prompt_classify = PromptTemplate.from_template("""Given the user question below, classify it as either being about
 `Physics`, `Math`, `Computer Science`. Do not respond with more than one word.
 <question> {question} </question> Classification:""")

In [ ]:
prompt_classify

In [ ]:
# Create a chain that combines the PromptTemplate, language model, and output parser
chain = prompt_classify | llm | StrOutputParser()

# Invoke the chain with a sample question about the first law of motion
chain.invoke({'question': "what is first law of motion?"})

In [ ]:
chain.invoke({"question":"What's a programming language?"})

In [ ]:
# Template for physics-related questions and responses
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{question}
Start your answer with `Subject:Physics :`
"""

# Create a ChatPromptTemplate using the physics template
physics_prompt = ChatPromptTemplate.from_template(physics_template)

# Chain the template, language model, and output parser
physics_chain = physics_prompt | llm | StrOutputParser()

In [ ]:
# Template for math-related questions and responses
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{question}
Start your answer with `Subject:Math :`
"""

# Create a ChatPromptTemplate using the math template
math_prompt = ChatPromptTemplate.from_template(math_template)

# Chain the template, language model, and output parser
math_chain = math_prompt | llm | StrOutputParser()

In [ ]:
# Template for computer science-related questions and responses
computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{question}
Start your answer with `Subject:Computer Science :`
"""

# Create a ChatPromptTemplate using the computer science template
computer_science_prompt = ChatPromptTemplate.from_template(computerscience_template)

# Chain the template, language model, and output parser
computer_science_chain = computer_science_prompt | llm | StrOutputParser()

In [ ]:
# Template for general-purpose questions and responses
general_prompt = ChatPromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Start your answer with `Subject:General :`"""
)

# Chain the template, language model, and output parser
general_chain = general_prompt | llm | StrOutputParser()

In [ ]:
def route(info):
    # Check if the word 'physics' is in the topic, case-insensitive.
    if "physics" in info["topic"].lower():
        return physics_chain  # Return the physics-specific chain if topic is related to physics.

    # Check if the word 'math' is in the topic, case-insensitive.
    elif "math" in info["topic"].lower():
        return math_chain  # Return the math-specific chain if topic is related to math.

    # Check if the phrase 'computer science' is in the topic, case-insensitive.
    elif "computer science" in info["topic"].lower():
        return computer_science_chain  # Return the computer science-specific chain if topic is related to computer science.

    # If none of the above topics are found, return the general-purpose chain.
    else:
        return general_chain  # Return the general-purpose chain for any other topics.

In [ ]:
from langchain_core.runnables import RunnableLambda

# Define a dictionary that contains the "topic" and "question".
# The "topic" maps to the appropriate chain, while the "question" is a lambda function
# that extracts the "question" from the input data.
full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)

# Display the full chain pipeline
full_chain

In [ ]:
full_chain.invoke({"question":"What is black body radiation?"})

In [ ]:
full_chain.invoke({"question":"What's a programming language?"})

In [ ]:
full_chain.invoke({"question":"What's 1+1"})

In [ ]:
full_chain.invoke({"question":"What's a path integral?"}) #

In [ ]:
full_chain.invoke({"question":"tell me 2 facts about Taj Mahal India?"})